In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install datasets transformers

In [ ]:
!pip install flask-ngrok
!pip install flask-bootstrap

In [ ]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf  /content/ngrok-v3-stable-linux-amd64.tgz

In [ ]:
! /content/ngrok config add-authtoken 2OdU9FiLDxahD6uzIKXWPgWOAL2_6gXeqwNHHe5vHcJTGrxer

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=d9a73ae0a74fff83054c2e8aa929dcb3f6276bad9bb20ad593969a49280eebd9
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
batch_size = 1

In [ ]:
# Some model classes

'''ResNet in PyTorch.

For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=2):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=(3,3), stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=(3,3),
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=14):
        super(ResNet, self).__init__()
        self.in_planes = 3

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=2)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        #self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        #out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(x)
        dl1 = out.shape
        out = self.layer2(out)
        dl2 = out.shape
        out = self.layer3(out)
        dl3 = out.shape
        out = self.layer4(out)
        dl4 = out.shape
        #out = F.avg_pool2d(out, 4)
        #out = out.view(out.size(0), -1)

        # Flatten (batch size, channels, features)
        out = out.view(batch_size, 512, 140)
        out_1_s = out.shape
        # Might have to change the last layer to obtain our 14x11 feature vector?
        #out = self.linear(out)
        #print(f'out_1_s: {out_1_s}, out: {out.shape}')
        #print(f'dl1: {dl1}, dl2: {dl2}, dl3: {dl3}, dl4: {dl4}, out: {out_1_s}')
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

# ----------------------------------
# Cross Attention Model for Language
class CrossAttentionModelLanguage(nn.Module):
    # Changed hidden_dim from 140 to 1
    def __init__(self, input_dim=64, hidden_dim=1, num_heads=4):
        super(CrossAttentionModelLanguage, self).__init__()

        self.attention = nn.MultiheadAttention(input_dim, num_heads)
        self.linear = nn.Linear(input_dim, hidden_dim)

    def forward(self, input):
        # Reshape the input to (sequence_length, batch_size, input_dim)
        input = input.permute(1, 0, 2)

        # Apply cross-attention
        output, _ = self.attention(input, input, input)

        # Reshape the output to (batch_size, sequence_length, input_dim)
        output = output.permute(1, 0, 2)

        # Apply linear transformation
        output = self.linear(output)

        return output


#--------------------------------
# Multimodal Transformer Network
from torch.nn import Transformer

class TransformerNetwork(nn.Module):
    def __init__(self):
        super(TransformerNetwork, self).__init__()

        self.embedding_dim = 512
        self.num_layers = 8
        self.num_heads = 8

        self.embedding = nn.Linear(142, self.embedding_dim)
        self.transformer = Transformer(
            d_model=self.embedding_dim,
            nhead=self.num_heads,
            num_encoder_layers=self.num_layers,
            num_decoder_layers=self.num_layers
        )
        #self.output_layer = nn.Linear(self.embedding_dim, 1)

    def forward(self, input_tensor):
        batch_size = input_tensor.size(0)

        embedded = self.embedding(input_tensor)
        embedded = embedded.permute(1, 0, 2)

        output = self.transformer(embedded, embedded)
        output = output.permute(1, 0, 2)
        #output = self.output_layer(output)

        return output.squeeze(2)

#-------------------
# LSTM Netowkr Part
class TwoLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(TwoLSTM, self).__init__()
        self.hidden_size = hidden_size

        # Note: we use a Linear layer for dimension adjustment.
        # Indeed, the previous multi-modal transformer outputs a 1024 + prev_action length tensor,
        # and here the LSTM layers have a 512 dim where we also need to bypass residual connections.
        # We use the linear layer below to have a matching size with the input/output of these LSTMs
        # in order to concatenate them.
        self.linear = nn.Linear(input_size, hidden_size)

        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=2, batch_first=True)

        # This linear layer is for the final mapping
        #self.fc = nn.Linear(hidden_size, output_size) #fully connected last layer

    def forward(self, input):
        # Perform dimension adjustment using the linear layer
        adjusted_input = self.linear(input)
        #print(f'adjusted_input: {adjusted_input.shape}')

        # Perform the forward pass through each LSTM layer
        output, _hidden_state = self.lstm(adjusted_input)

        # Perform residual connections between LSTM layers
        residual_output = adjusted_input + output

        # Extract the hidden state of the last LSTM layer after residual connections
        last_hidden_state = residual_output[:, -1, :]

        #out = self.fc(last_hidden_state)
        return last_hidden_state

In [ ]:

import torch.nn as nn
import torch
import torch.nn.functional as F

class CCNeT5(nn.Module):
    def __init__(self):
        super().__init__()
        #self.rgb_model = nn.Sequential(*resnet_blocks)
        self.rgb_model = ResNet18()
        self.language_model = CrossAttentionModelLanguage()

        # Multi Modal Part Combining RGB + Language
        #self.multimodal_transformer = MultiModalTransformer()
        self.multimodal_transformer = TransformerNetwork()

        # Here we should concatenate with the previous Action
        # when doing the forward pass

        # Two-Layer LSTM
        self.lstm = TwoLSTM(1089, 513)

        # Last output Layer to create a proper action space from the original language embeddings
        self.fc = nn.Linear(513, 577) #fully connected last layer

        # T5 Decreased
        # Linear layer to reduce the size of the T5 output frm 577 to 512
        self.fc_t5 = nn.Linear(577, 512)


    # todo: pass previous action
    def forward(self, rgb_input, language_input, previous_action, t5_output):
        # Process RGB input
        rgb_output = self.rgb_model(rgb_input)

        # Process language input
        language_output = self.language_model(language_input)

        #print(f'shape rbg_output: {rgb_output.shape}, shape language_output: {language_output.shape}')
        flattened_tensor1 = rgb_output.view(batch_size, 512*140)
        flattened_tensor2 = language_output.view(batch_size, -1)

        # Concatenate tensor1 and flattened_tensor2 along the last dimension
        fused_output = torch.cat((flattened_tensor1, flattened_tensor2), dim=1).view(batch_size, 512, 141)

        # Add T5 output to image and language tensor
        t5_output = self.fc_t5(t5_output).unsqueeze(dim=2) # Change shape from (batch, 577) to (batch, 512, 1)
        # Concatenate with fused output
        fused_output = torch.cat((fused_output, t5_output), dim=2)

        # Further processing or fusion of modalities
        #print(f'fused_output: {fused_output.shape} - {fused_output.dtype}')
        # Additional processing steps...

        # Permute
        #fused_output = fused_output.permute(0, 2, 1)

        # Feed into Multimodal Transformer
        multimodal_output = self.multimodal_transformer(fused_output)
        #print(f'multimodal_output: {multimodal_output.shape} - type: {multimodal_output.dtype}')

        # TODO: concatenate with previous action
        #print(f'previous_action: {previous_action.shape}')
        #tensor2_reshaped = F.pad(previous_action, (0, 447), mode='constant')
        #tensor2_expanded = torch.unsqueeze(tensor2_reshaped, dim=2)  # Shape: [8, 1024, 577]
        #combined_tensor = torch.cat((multimodal_output, tensor2_expanded), dim=2).type(torch.float32) # Shape: [8, 1024, 1089]

        tensor2 = previous_action.unsqueeze(1)
        # Expand tensor2 dimensions to match tensor1
        tensor2 = tensor2.expand(-1, 512, -1)
        # Combine the tensors
        combined_tensor = torch.cat((multimodal_output, tensor2), dim=2).type(torch.float32)
        #print(f'concatenation previous action: {combined_tensor.shape}')

        # Feed into LSTM
        # + Permute to fit the right format
        #lstm_output = self.lstm(combined_tensor.permute(0, 2, 1))
        lstm_output = self.lstm(combined_tensor)
        #print(f'lstm_output: {lstm_output.shape}')

        # Last layer to target action space
        action_output = self.fc(lstm_output)

        return action_output


In [ ]:
# 1 - Get out Tokenizer Class
css_fields = ['top', 'left', 'width', 'height']
special_characters = ['.', ',', '#', ':', '-', '/', '(', ')', 'https://', '@', '&', '"', "'", '!', '?', ';', '+', '=',
                      '*', '$', '€', '*', '`']


def round_to_nearest_ten(number):
    return round(number / 10) * 10

import torch
import torch.nn as nn


# Turn it into a class
class CCNeT5Tokenizer:
    def __init__(self, vocab_path):
        stoi = {}
        itos = {}
        self.padding_char = '<PAD>'
        self.special_characters = ['.', ',', '#', ':', '-', '/', '(', ')', 'https://', '@', '&', '"', "'", '!', '?',
                                   ';', '+', '=', '*', '$', '€', '*', '`']

        with open(vocab_path, 'r') as file:
            for index, line in enumerate(file):
                line = line.strip()
                stoi[line] = index
                itos[index] = line

        stoi[' '] = stoi['']
        itos[stoi[' ']] = ' '
        self.stoi = stoi
        self.stoi[self.padding_char] = len(stoi.keys())-1  # Add PADDING character
        # We do not need itos as we don't implement a de-tokinezing function.
        self.itos = itos
        self.itos[len(stoi.keys()) - 2] = self.padding_char  # Add PADDING character
        print(f'pad: {len(stoi.keys()) - 2}')
        print(f'Loaded CC_NeT5 Tokenizer with vocabulary size being {len(self.stoi)}.')

        # Instantiate the embedding function
        vocab_size = len(stoi)

    # Provide a string and tokenize it: utterance or task name.
    def tokenize_string(self, string):
        tokenized_string = []
        for w in str(string).lower().split(' '):
            for sc in self.special_characters:
                if sc in w:
                    w = w.replace(sc, 'Ø' + sc + 'Ø')
            p1_words = [s for s in w.split('Ø') if s != '']
            # Check if the found words exist in our vocab, else subdivide
            for fw in p1_words:
                if fw not in self.stoi.keys():
                    for c in fw:
                        tokenized_string.append(self.stoi[c])
                else:
                    tokenized_string.append(self.stoi[fw])
        return tokenized_string

    # Turns an array into a string
    def detokenize_array(self, array):
        reconstruted_string = []
        #pad_index = self.itos[self.stoi[self.padding_char]]
        for v in array:
            v = int(v)
            s = self.itos[v]
            if s != self.padding_char:
                reconstruted_string.append(s)

        #reconstruted_string = ' '.join(reconstruted_string)
        cleaned_string = ''
        last_single = False
        for s in reconstruted_string:
            if len(s) > 1:
                if last_single:
                    cleaned_string += ' '
                    last_single = False
                cleaned_string += s + ' '
            else:
                # We do this to treat single length characters
                cleaned_string += s
                last_single = True
        return cleaned_string

    # Truncate and pad the incoming sentences based on a max_size argument
    def truncate_pad_entry(self, tokenized_array, max_size):
        if len(tokenized_array) < max_size:
            while len(tokenized_array) < max_size:
                tokenized_array.append(self.stoi[self.padding_char])
            return tokenized_array
        elif len(tokenized_array) > max_size:
            # Too long, so we automatically truncate
            return tokenized_array[:max_size]
        else:
            # Just avoid iterating under the hood
            return tokenized_array

    # Tokenize a DOM dictionary
    def tokenize_dom(self, dom):
        tokenized_dom = []

        # Add opening tag
        element_tag = dom['tag'].lower()
        if 'input' in dom.keys():
            # if tag is input, reformat it
            element_tag = dom['input'].lower() + '_' + dom['type'].lower()
            tokenized_dom.append(self.stoi['<' + element_tag])
        else:
            # add normal tag
            tokenized_dom.append(self.stoi['<' + dom['tag'].lower()])

        for field in dom:
            if field != 'tag' and field != 'children' and field != 'type' and field != 'text' and field not in css_fields:  # and field != 'value':

                tokenized_dom.append(self.stoi[field.lower()])

                # Ensure we don't have float values, we'll stick with integers
                if isinstance(dom[field], float):
                    tokenized_dom.append(self.stoi[str(int(round_to_nearest_ten(dom[field])))])
                else:
                    words = str(dom[field]).lower().split(' ')
                    for word in words:
                        for sc in special_characters:
                            if sc in word:
                                word = word.replace(sc, 'Ø' + sc + 'Ø')
                        p1_words = [s for s in word.split('Ø') if s != '']

                        # decides whether we keep the full word, or just the letters:
                        if field == 'value' or (False and (field == 'label' or field == 'button')):
                            for w in p1_words:
                                # Take individual characters of the value string
                                processed_words = [self.stoi[w[i:i + 1]] for i in range(0, len(w), 1)]
                                tokenized_dom += processed_words

                        elif field == 'ref':
                            for w in p1_words:
                                # Take individual characters of the value string
                                processed_words = [self.stoi[w[i:i + 3]] for i in range(0, len(w), 3)]
                                tokenized_dom += processed_words
                        else:
                            # Use the full word:
                            for w in p1_words:
                                tokenized_dom.append(self.stoi[str(w)])

            elif field == 'text':
                tokenized_dom.append(self.stoi['text'])
                for c in dom[field]:
                    tokenized_dom.append(self.stoi[c])
            elif field in css_fields:
                # Cast field
                tokenized_dom.append(self.stoi[field])
                css_value = int(round_to_nearest_ten(float(dom[field])))
                tokenized_dom.append(self.stoi[str(css_value)])

        if 'children' in dom.keys():
            tokenized_dom.append(self.stoi['children'])
            for child in dom['children']:
                tokenized_dom += self.tokenize_dom(child)
                # for v in found_vocab:
                #    tokenized_dom.append(self.stoi[v])

        # add closing tag
        tokenized_dom.append(self.stoi['</' + element_tag + '>'])

        return tokenized_dom

vocab_path="/content/drive/MyDrive/WebAI/Notebooks/CC_NeT5/vocab.txt"
embedding_fn_path = '/content/drive/MyDrive/WebAI/Configs/embedding_weights.pth'
tokenizer = CCNeT5Tokenizer(vocab_path=vocab_path)

pad: 1590
Loaded CC_NeT5 Tokenizer with vocabulary size being 1592.


In [ ]:


# Load some of the images we have here and try to save them to the dataset
import os
from typing import List, Tuple
import numpy as np
from PIL import Image
import torch
import pandas as pd
import re
import json
from torch.utils.data import DataLoader
import random
import math

class DatasetLoader:
    #def __init__(self, screenshots_path, dataset, start_index, end_index, dom_tokenizer, batch_size):
    def __init__(self, screenshots_path, dataset, dom_tokenizer, batch_size):

        # Get the Subset of the dataset
        #self.start_index = start_index
        #self.end_index = end_index
        self.df = dataset['train'].to_pandas()##[self.start_index:self.end_index]
        # When creating the class, parse JSON files into a state-based list structure
        self.df['processed_states'] = self.df['processed_states'].apply(lambda x: json.loads(re.sub(r'\b(True|False)\b', lambda m: m.group(0).lower(), x.replace("'", '"'))))
        print(f'Dataset subset has {len(self.df)} rows')

        self.screenshots_path = screenshots_path
        # If want to load them from the directory
        self.indices, found_screenshots = self.get_file_indices()
        # Or load the previously saved indices
        #self.indices = self.read_indices_from_file('/content/drive/MyDrive/WebAI/file_indices.txt')
        found_screenshots = len(self.indices)
        print(f'Found a total of {found_screenshots} state screenshots for {len(self.indices)} episodes.')

        # Load all images from the drive
        #self.add_images_to_episodes()
        # Load N samples

        self.dom_tokenizer = dom_tokenizer
        self.dom_seq_length = 492
        self.utterance_seq_length = 16
        self.task_name_seq_length = 4

        self.target_action_type_seq_length = 1
        self.target_ref_seq_length = 1
        self.target_keydown_seq_length = 8 #32

        # Size of the final layer output being composed of
        # 1 (action),
        # + 64 (ref) (
        # + 64x8 (keydown text times target_keydown_seq_length)
        self.model_output_size = 577

        self.batch_size = batch_size

        self.max_ref_nb = 500

    # Add the images to the different episodes of the dataset
    def add_images_to_episodes_old(self):
        found_images = []
        i = self.start_index
        image_counter = 0
        while i < self.end_index:
            if i % 100 == 0:
              print(f'Done: {i}')
            # Get K indices
            images = []
            try:
              state_indexes = self.indices[i]
              for state_index in state_indexes:
                  images.append(self.read_image(i, state_index))
                  image_counter += 1
              found_images.append(images)
            except:
              print(f'error {i}')
            i += 1
        # Add new column
        self.df['episode_images'] = found_images
        print(f'Found total of {image_counter} images in the loaded episodes.')

    # Load the images in batches
    def add_images_to_episodes(self):
      found_images = []
      image_counter = 0

      for i in range(self.start_index, self.end_index):
        if i%100 == 0:
          print(f'Done :{i}')
        try:
            state_indexes = self.indices[i]
            images = [self.read_image(i, state_index) for state_index in state_indexes]
            found_images.append(images)
            image_counter += len(images)
        except Exception as e:
            print(f'Error at index {i}: {str(e)}')
            # FFUUUCK NEED TO DO SOMETHING HERE IF ERROR...
            state_indexes = self.indices[i]
            for state_index in state_indexes:
              try:
                images.append(self.read_image(i, state_index))
                image_counter += 1
              except:
                'ehhh'
            found_images.append(images)


      self.df['episode_images'] = found_images
      print(f'Found a total of {image_counter} images in the loaded episodes.')


    def get_file_indices(self) -> List[Tuple[int, int]]:
        indices = []
        for filename in os.listdir(self.screenshots_path):
            if filename.endswith('.png'):
                # Extract the N and K values from the filename
                N, K = filename.replace('sc_', '').replace('st_', '').replace('.png', '').split('_')
                N = int(N)
                K = int(K)
                indices.append((N, K))

        # Turn the indices into a dictionary
        indices_dict = {}
        for (N, K) in indices:
            if N not in indices_dict:
                indices_dict[N] = [K]
            else:
                indices_dict[N].append(K)
                indices_dict[N] = sorted(indices_dict[N])

        return indices_dict, len(indices)

    def save_file_indices_to_file(self, dictionary, filename):
      with open(filename, 'w') as file:
          for key, value in dictionary.items():
              line = f"{key}: {value}\n"
              file.write(line)

    def read_indices_from_file(self, filename):
        dictionary = {}
        with open(filename, 'r') as file:
            for line in file:
                key, value = line.strip().split(': ')
                key = int(key)
                value = [int(num) for num in value.strip('[]').split(', ')]
                dictionary[key] = value
        return dictionary

    import torch

    # Reads images into tensor
    def read_image_old(self, N, K) -> torch.Tensor:
        filename = f'sc_{N}_st_{K}.png'
        filepath = os.path.join(self.screenshots_path, filename)
        image = Image.open(filepath).convert('RGB')
        tensor = torch.tensor(np.array(image), dtype=torch.float32).permute(2, 0, 1) / 255.0
        return tensor

    # Pure NP instead of tensor
    def read_image(self, N, K) -> np.array:
        filename = f'sc_{N}_st_{K}.png'
        filepath = os.path.join(self.screenshots_path, filename)
        image = Image.open(filepath).convert('RGB')
        array = np.transpose(np.array(image) / 255.0, (2, 0, 1))
        return array


    def save_image_png(self, N, K):
        loaded_image = self.read_image(N, K).numpy()
        print(f'Image has shape {loaded_image.shape}')

        # scale the pixel values from [0,1] to [0, 255]
        array = np.clip(loaded_image * 255, 0, 255).astype('uint8')
        img = Image.fromarray(array.transpose(1, 2, 0), mode='RGB')
        img.save('screenshots/test_img.png')

    # Turn our dataset into a series of state tensors ready for training.
    # We though ensure to clearly separate our different instances:
    # - todo: T5-Data (for later, depends between SL or RL current approach)
    # - RBG
    # - todo: Tokenized DOM
    # - todo: Task Instruction
    # They are separated because they are not going to be fed in the same manner into the model.
    # Basically into one single row.
    # todo: skip rows that have negative rewards
    def process_dataset(self):
        rbg_data = []
        dom_data = []
        utterance_data = []
        task_name_data = []

        target_action = []
        target_refs = []
        target_keydown = []

        episode_previous_actions = []

        # Some metrics
        duplicated_images = 0
        cut_images = 0

        for (index, row) in self.df.iterrows():
            if float(row['reward']) <= 0: # Skips failed episodes
                continue
            if len(row['episode_images']) != len(row['processed_states']): # Skips episodes with no matching pictures amount and states
                len_processed_states = len(row['processed_states'])
                len_episode_images = len(row['episode_images'])
                print(f'Error: episode_images and processed_states don\' have the same length for index {index}: len_episode_images={len_episode_images}, len_processed_states={len_processed_states}')
                # We just discard it, fuck it if it's just an individual row and only one image missing out of n states.
                # Data processing showed that only few states were missing from the zip and got fixed except a couple of them

                # Ok actually fix it by cloning the last rgb
                if len_episode_images < len_processed_states:
                  last_img = row['episode_images'][len(row['episode_images'])-1]
                  while len(row['episode_images']) < len_processed_states:
                    row['episode_images'].append(last_img)
                    duplicated_images += 1
                else:
                  # Too few processed states for pictures, cut last pic.
                  # We keep the processed states over the generated screenshots
                  row['episode_images'] = row['episode_images'][:len_processed_states]
                  cut_images += len_episode_images - len_processed_states
                #continue
            for rbg in row['episode_images']:
                rbg = np.array(rbg, dtype='float64')
                #print(f'rbg shape: {rbg.shape}')
                rbg_data.append(rbg)


            # Process DOM Data
            for (index_state, state) in enumerate(row['processed_states']):

                # Output Tokens
                if state['action_type'] == 'click': # Append boolean for action click or keydown
                    t_action = np.array([0])
                    target_action.append(t_action)
                else:
                    t_action = np.array([1])
                    target_action.append(t_action)

                # Tokenize ref and appends to target
                #sr = state['refs']
                #print(f'state ref: {sr})
                #t_ref = np.array(self.dom_tokenizer.truncate_pad_entry(self.dom_tokenizer.tokenize_string(str(state['refs'])), self.target_ref_seq_length))
                # NEW: don't tokenize with ITOS, pass ref value directly to reduce the vocab size
                if abs(int(state['refs'])) >= self.max_ref_nb:
                  state_ref = self.max_ref_nb-1
                else:
                  state_ref = abs(int(state['refs']))
                if state_ref < 0:
                  print(f'ERRROR state_ref: {state_ref}')
                  return
                t_ref = np.array(self.dom_tokenizer.truncate_pad_entry([state_ref], self.target_ref_seq_length))
                target_refs.append(t_ref)

                # Tokenize target text and appends to target
                t_keydown = np.array(self.dom_tokenizer.truncate_pad_entry(self.dom_tokenizer.tokenize_string(state['keydown_text']), self.target_keydown_seq_length))
                target_keydown.append(t_keydown)

                # Input Tokens
                # Tokenize dom
                tokenized_dom = self.dom_tokenizer.tokenize_dom(state['dom'])
                tokenized_dom = self.dom_tokenizer.truncate_pad_entry(tokenized_dom, self.dom_seq_length)
                dom_data.append(tokenized_dom)

                # Tokenize utterance
                tokenized_utterance = self.dom_tokenizer.truncate_pad_entry(self.dom_tokenizer.tokenize_string(row['utterance']), self.utterance_seq_length)
                utterance_data.append(tokenized_utterance)

                # Tokenize task name
                tokenized_task_name = self.dom_tokenizer.truncate_pad_entry(self.dom_tokenizer.tokenize_string(row['task_name']), self.task_name_seq_length)
                task_name_data.append(tokenized_task_name)

                # Deal with the previous action
                if index_state < len(row['processed_states'])-1:
                    # Default one so empty for the first action
                    if index_state == 0:
                        episode_previous_actions.append([])
                        #episode_previous_actions.append(self.dom_tokenizer.truncate_pad_entry(self.dom_tokenizer.tokenize_string(''), self.model_output_size))

                    # Add previous actions one by one
                    episode_previous_actions.append([t_action, t_ref, t_keydown])
                elif index_state >= len(row['processed_states'])-1 and index_state == 0:
                    episode_previous_actions.append([])
                # Else no action to add




        rbg_data = np.stack(rbg_data)
        dom_data = np.array(dom_data)
        utterance_data = np.array(utterance_data)
        task_name_data = np.array(task_name_data)
        target_action = np.array(target_action)
        target_refs = np.array(target_refs)
        target_keydown = np.array(target_keydown)

        print(f'duplicated_images: {duplicated_images}, cut_images: {cut_images}')

        return rbg_data, dom_data, utterance_data, task_name_data, target_action, target_refs, target_keydown, episode_previous_actions


    # Batchify the dataset
    def get_dataloder(self, dataset):
        train_loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        return train_loader

    # Creates random indexes to split and shuffle the test and training datasets
    def create_train_test_dataset(self, length):
        # Calculate the number of random indexes (10% of length)
        num_random_indexes = int(length * 0.1)

        # Generate random indexes without replacement
        random_indexes = random.sample(range(length), num_random_indexes)

        # Create train and test datasets
        train_dataset = []
        test_dataset = []

        for i in range(length):
            if i in random_indexes:
                test_dataset.append(i)
            else:
                train_dataset.append(i)

        # Shuffle the train and test datasets
        random.shuffle(train_dataset)
        random.shuffle(test_dataset)

        return train_dataset, test_dataset

    # Devise datasets
    def split_datasets(self, rbg_data, dom_data, utterance_data, task_name_data, target_action, target_refs, target_keydown, episode_previous_actions):
        train_indexes, test_indexes = self.create_train_test_dataset(length=len(rbg_data))

        train_rgb = rbg_data[train_indexes]
        train_dom = dom_data[train_indexes]
        train_utterance = utterance_data[train_indexes]
        train_task_name = task_name_data[train_indexes]
        train_target_action = target_action[train_indexes]
        train_target_ref = target_refs[train_indexes]
        train_target_keydown = target_keydown[train_indexes]
        train_episode_previous_action = []
        for index in train_indexes:
            train_episode_previous_action.append(episode_previous_actions[index])

        # Create a boolean mask of the indexes
        test_rgb = rbg_data[test_indexes]
        test_dom = dom_data[test_indexes]
        test_utterance = utterance_data[test_indexes]
        test_task_name = task_name_data[test_indexes]
        test_target_action = target_action[test_indexes]
        test_target_ref = target_refs[test_indexes]
        test_target_keydown = target_keydown[test_indexes]
        test_episode_previous_action = []
        for index in test_indexes:
            test_episode_previous_action.append(episode_previous_actions[index])

        return train_rgb, train_dom, train_utterance, train_task_name, train_target_action, train_target_ref, \
               train_target_keydown, train_episode_previous_action, test_rgb, test_dom, test_utterance, \
               test_task_name, test_target_action, test_target_ref, test_target_keydown, test_episode_previous_action

    # Get a dictionary with each task name, load N samples per task
    def get_predefined_indices(self, path, N, dataset):
      def load_dictionary_from_file(file_path):
        with open(file_path, 'r') as file:
            dictionary = json.load(file)
            return dictionary
      dict_tasks = load_dictionary_from_file(path)
      print(dict_tasks)

      import random

      def sample_N(x, N):
        if len(x) < N:
          return x
        return random.sample(x, N)

      downsampled_tasks = {}
      downsampled_tasks_counts = {}
      total_counts = 0
      for key, value in dict_tasks.items():
        samples = sample_N(value, N)
        total_counts += len(samples)
        downsampled_tasks[key] = sorted(samples)
        downsampled_tasks_counts[key] = len(samples)

      print(f'Got total downsampled_tasks_counts: {total_counts}')
      print(f'downsampled_tasks: {downsampled_tasks}')

      # Put them into a single list
      indices = []
      for key, samples in downsampled_tasks.items():
        #print(f'samples: {samples}')
        indices += samples
      indices = sorted(indices)
      print(f'found indices: {indices}')
      print(f'self indices: {self.indices}')

      # Now load corresponding K values for RGB data from the original loaded indices
      indices_to_load = {}
      failed_sample_indexes = []
      for index in indices:
        try:
          samples = self.indices[index]
          #if index not in indices_to_load.keys():
            #indices_to_load[index] = []
          indices_to_load[index] = samples
        except Exception as e:
          print(f'Could not load samples at index {index}: {e}')
          failed_sample_indexes.append(index)
      print(f'failed_sample_indexes: {failed_sample_indexes}')

      found_images = []
      image_counter = 0
      # Now load RGB pictures
      i = 0

      # List our indices in order:
      ordered_indices = sorted(indices_to_load.keys())
      print(f'indices_to_load: {indices_to_load}')
      print(f'ordered_indices: {ordered_indices}')

      for row_index in ordered_indices:
        state_indexes = indices_to_load[row_index]
        #print(f'state_indexes: {row_index}-{state_indexes}')
        if i % 50 == 0:
          print(f'Done: {i}')
        loaded_images_episode = 0
        episode_images = []

        try:
          images = [self.read_image(row_index, state_index) for state_index in state_indexes]
          image_counter += len(images)
          loaded_images_episode += len(images)
          episode_images = images
        except Exception as e:
          episode_images = []
          print(f'Error at index {i}: {str(e)}')
          # FFUUUCK NEED TO DO SOMETHING HERE IF ERROR...
          for state_index in state_indexes:
            try:
              episode_images.append(self.read_image(row_index, state_index))
              image_counter += 1
              loaded_images_episode += 1
            except:
              'ehhh'
        found_images.append(episode_images)
        if loaded_images_episode != len(state_indexes):
          print(f'Not same length loaded_images_episode: {loaded_images_episode} - {len(state_indexes)} for index {row_index}')
        if len(episode_images) != len(state_indexes):
          print(f'Not same length episode_images: {len(episode_images)} - {len(state_indexes)} for index {row_index}')

        i += 1


      print(f'Found a total of {image_counter} images in the loaded episodes.')


      # Reformat our self.df with the selected rows
      self.df = dataset.iloc[ordered_indices]
      # When creating the class, parse JSON files into a state-based list structure
      self.df['processed_states'] = self.df['processed_states'].apply(lambda x: json.loads(re.sub(r'\b(True|False)\b', lambda m: m.group(0).lower(), x.replace("'", '"'))))
      print(f'Dataset subset sampled has {len(self.df)} rows')
      print(f'found_images for total episodes: {len(found_images)}')
      self.df.loc[:, 'episode_images'] = found_images

In [ ]:
!git clone https://github.com/LucasStill/miniwob_zip.git

["Cloning into 'miniwob_zip'...",
 'remote: Enumerating objects: 24148, done.\x1b[K',
 'remote: Counting objects:   0% (1/603)\x1b[K',
 'remote: Counting objects:   1% (7/603)\x1b[K',
 'remote: Counting objects:   2% (13/603)\x1b[K',
 'remote: Counting objects:   3% (19/603)\x1b[K',
 'remote: Counting objects:   4% (25/603)\x1b[K',
 'remote: Counting objects:   5% (31/603)\x1b[K',
 'remote: Counting objects:   6% (37/603)\x1b[K',
 'remote: Counting objects:   7% (43/603)\x1b[K',
 'remote: Counting objects:   8% (49/603)\x1b[K',
 'remote: Counting objects:   9% (55/603)\x1b[K',
 'remote: Counting objects:  10% (61/603)\x1b[K',
 'remote: Counting objects:  11% (67/603)\x1b[K',
 'remote: Counting objects:  12% (73/603)\x1b[K',
 'remote: Counting objects:  13% (79/603)\x1b[K',
 'remote: Counting objects:  14% (85/603)\x1b[K',
 'remote: Counting objects:  15% (91/603)\x1b[K',
 'remote: Counting objects:  16% (97/603)\x1b[K',
 'remote: Counting objects:  17% (103/603)\x1b[K',
 'remote: Count

In [ ]:
import numpy as np
from datasets import load_dataset
dataset = load_dataset("LucasThil/randomized_clean_miniwob_episodes_v2")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/13412 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/LucasThil___parquet/LucasThil--randomized_clean_miniwob_episodes_v2-49501296fc0102ba/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#screenshots_path = '/content/drive/MyDrive/WebAI/screenshot_indexes'
#screenshots_path = '/content/sample_data_screenshots'
screenshots_path = '/content/miniwob_zip'
batch_size = 1
# NOTE!!!: Check if need to rerun the indices on all pictures instead of range
#dataset_loader = DatasetLoader(screenshots_path=screenshots_path, dataset=dataset, start_index=0, end_index=20, dom_tokenizer=tokenizer, batch_size=batch_size)
dataset_loader = DatasetLoader(screenshots_path=screenshots_path, dataset=dataset, dom_tokenizer=tokenizer, batch_size=batch_size)

Dataset subset has 13412 rows
Found a total of 9184 state screenshots for 9184 episodes.


In [ ]:

# Some model classes

'''ResNet in PyTorch.

For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=2):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=(3,3), stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=(3,3),
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=14):
        super(ResNet, self).__init__()
        self.in_planes = 3

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=2)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        #self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        #out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(x)
        dl1 = out.shape
        out = self.layer2(out)
        dl2 = out.shape
        out = self.layer3(out)
        dl3 = out.shape
        out = self.layer4(out)
        dl4 = out.shape
        #out = F.avg_pool2d(out, 4)
        #out = out.view(out.size(0), -1)

        # Flatten (batch size, channels, features)
        out = out.view(batch_size, 512, 140)
        out_1_s = out.shape
        # Might have to change the last layer to obtain our 14x11 feature vector?
        #out = self.linear(out)
        #print(f'out_1_s: {out_1_s}, out: {out.shape}')
        #print(f'dl1: {dl1}, dl2: {dl2}, dl3: {dl3}, dl4: {dl4}, out: {out_1_s}')
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

# ----------------------------------
# Cross Attention Model for Language
class CrossAttentionModelLanguage(nn.Module):
    # Bigger2: Changed hidden_dim from 140 to 1
    def __init__(self, input_dim=64, hidden_dim=140, num_heads=4):
        super(CrossAttentionModelLanguage, self).__init__()

        self.attention = nn.MultiheadAttention(input_dim, num_heads)
        self.linear = nn.Linear(input_dim, hidden_dim)

    def forward(self, input):
        # Reshape the input to (sequence_length, batch_size, input_dim)
        input = input.permute(1, 0, 2)

        # Apply cross-attention
        output, _ = self.attention(input, input, input)

        # Reshape the output to (batch_size, sequence_length, input_dim)
        output = output.permute(1, 0, 2)

        # Apply linear transformation
        output = self.linear(output)

        return output


#--------------------------------
# Multimodal Transformer Network
from torch.nn import Transformer

class TransformerNetwork(nn.Module):
    def __init__(self):
        super(TransformerNetwork, self).__init__()

        self.embedding_dim = 512
        self.num_layers = 1
        self.num_heads = 8

        # Bigger2: 512, self.embedding_dim)# version1: 141
        self.embedding = nn.Linear(512, self.embedding_dim)
        self.transformer = Transformer(
            d_model=self.embedding_dim,
            nhead=self.num_heads,
            num_encoder_layers=self.num_layers,
            num_decoder_layers=self.num_layers
        )
        #self.output_layer = nn.Linear(self.embedding_dim, 1)

    def forward(self, input_tensor):
        batch_size = input_tensor.size(0)

        embedded = self.embedding(input_tensor)
        embedded = embedded.permute(1, 0, 2)

        output = self.transformer(embedded, embedded)
        output = output.permute(1, 0, 2)
        #output = self.output_layer(output)

        return output.squeeze(2)

#-------------------
# LSTM Netowkr Part
class TwoLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(TwoLSTM, self).__init__()
        self.hidden_size = hidden_size

        # Note: we use a Linear layer for dimension adjustment.
        # Indeed, the previous multi-modal transformer outputs a 1024 + prev_action length tensor,
        # and here the LSTM layers have a 512 dim where we also need to bypass residual connections.
        # We use the linear layer below to have a matching size with the input/output of these LSTMs
        # in order to concatenate them.
        self.linear = nn.Linear(input_size, hidden_size)

        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=2, batch_first=True)

        # This linear layer is for the final mapping
        #self.fc = nn.Linear(hidden_size, output_size) #fully connected last layer

    def forward(self, input):
        # Perform dimension adjustment using the linear layer
        adjusted_input = self.linear(input)
        #print(f'adjusted_input: {adjusted_input.shape}')

        # Perform the forward pass through each LSTM layer
        output, _hidden_state = self.lstm(adjusted_input)

        # Perform residual connections between LSTM layers
        residual_output = adjusted_input + output

        # Extract the hidden state of the last LSTM layer after residual connections
        last_hidden_state = residual_output[:, -1, :]

        #out = self.fc(last_hidden_state)
        return last_hidden_state

In [ ]:
# Initialize default model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

multimodal = CCNeT5(vocab_size=len(tokenizer.itos))
# Or Load modal manually
path = f'/content/drive/MyDrive/WebAI/Model Weights/CCNeT5_OffLine1/CCNeT5_1_smaller_train_best.pth'
checkpoint = torch.load(path, map_location=torch.device(device))
multimodal.load_state_dict(checkpoint['model_state_dict'])
multimodal.to(device)

multimodal

CCNeT5(
  (embedding): Embedding(1591, 64)
  (rgb_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv2d(3, 32, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): BasicBlock(
        (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)

In [ ]:
def make_inference_model(model, rgb_input, language_input, previous_action_input, t5_output):

    rgb_rgb_input_data = rgb_input.to(device)
    language_input = language_input.to(device)
    print(rgb_rgb_input_data.dtype, language_input.dtype, previous_action_input.dtype, t5_output.dtype)

    action_output = model.forward(rgb_rgb_input_data, language_input, previous_action_input, t5_output)

    print(f'action_output: {action_output.shape} - {action_output.dtype}')

    return action_output

In [ ]:
#batch_size = 1
rgb_data_test = torch.randn((batch_size, 3, 160, 210)).to(device)
language_data_test = torch.rand((batch_size, 512, 64)).to(device)
t5_output = torch.randn([batch_size, 10]).to(device)
previous_action_input = torch.randn([batch_size, 10]).to(device)

print(rgb_data_test.shape, language_data_test.shape, t5_output.shape, previous_action_input.shape)
prediction = make_inference_model(multimodal, rgb_data_test, language_data_test, previous_action_input, t5_output)
print(f'prediction: {prediction.shape}')

torch.Size([1, 3, 160, 210]) torch.Size([1, 512, 64]) torch.Size([1, 10]) torch.Size([1, 10])
torch.float32 torch.float32 torch.float32 torch.float32


RuntimeError: ignored

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def find_closest_embedding(input_embedding, embedding_matrix):
    """
    Finds the index of the closest match to the input embedding based on cosine similarity.

    Args:
        input_embedding (np.ndarray): Array of shape (8,) representing the input embedding.
        embedding_matrix (np.ndarray): Array of shape (10, 8) representing the embedding matrix.

    Returns:
        int: Index of the closest match to the input embedding.
    """
    # Normalize input embedding for cosine similarity
    input_embedding_norm = input_embedding / np.linalg.norm(input_embedding)

    # Normalize embedding matrix for cosine similarity
    embedding_matrix_norm = embedding_matrix / np.linalg.norm(embedding_matrix, axis=1, keepdims=True)

    # Calculate cosine similarity between input embedding and embedding matrix
    similarity_scores = cosine_similarity(input_embedding_norm.reshape(1, -1), embedding_matrix_norm)

    # Find the index of the closest match
    closest_index = np.argmax(similarity_scores)

    return closest_index, similarity_scores

# Decode action, ref, keydown from output tensor (1, 577)
# Only decode by batch_size = 1
def decode_output(tokenizer, tensor):
  # Decode action_type into a string
  action_type = tensor[0, 0:1]
  if int(action_type.item()) == 0:
    action_type = 'click'
  else:
    action_type = 'keydown'

  embedded_ref = tensor[:, 1:65]
  decoded_ref_raw, _ = find_closest_embedding(embedded_ref, tokenizer.embedding_fn.embedding.weight.detach().numpy())
  print(f'decoded_ref_raw: {decoded_ref_raw}')
  decoded_ref = tokenizer.itos[decoded_ref_raw]

  keydown = tensor[0, 65:577].view(8, 64)
  keydowns = []
  for key in keydown:
      decoded_keydown, _ = find_closest_embedding(key, tokenizer.embedding_fn.embedding.weight.detach().numpy())
      keydowns.append(tokenizer.itos[decoded_keydown])
  keydowns = ''.join(keydowns).replace(tokenizer.padding_char, '')

  output = {'action_type': action_type, 'ref': decoded_ref, 'keydown': keydowns}
  return output

In [ ]:
# Decode output
decode_output(tokenizer, prediction.to('cpu').detach())

decoded_ref_raw: 83


{'action_type': 'click', 'ref': '1', 'keydown': ''}

# Actor Critic Model
If using V-MPO, you should load the Actor Critic weight of the model

In [ ]:
# Initialize default model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

actor_critic = CCNeT5()
# Or Load modal manually
path = f'/content/drive/MyDrive/WebAI/Model Weights/CCNeT5_OffLine1/CCNeT5_{0}_sampled_combined_loss.pth'
checkpoint = torch.load(path, map_location=torch.device(device))
actor_critic.load_state_dict(checkpoint['model_state_dict'])
actor_critic.to(device)

actor_critic

CCNeT5(
  (rgb_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv2d(3, 32, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): BasicBlock(
        (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchN

In [ ]:
import pickle
mse_loss_fn = nn.MSELoss()
import torch.optim as optim
optimizer = optim.AdamW(actor_critic.parameters(), lr=1e-4, weight_decay=1e-1, betas=(0.9, 0.99))

In [ ]:
previous_loss_value.item()

AttributeError: ignored

# Server Part

In [ ]:
# Define quick flask server to serve our endpoint
from flask import Flask
from pyngrok import ngrok
from flask import request, jsonify
import json

app = Flask(__name__)

@app.route('/')
def hello_world():
  return 'Hello World'


@app.route('/backprop_actor_critic', methods=['POST'])
def backprop_actor_critic():
    # Get the JSON payload from the request data
    # Parse the received JSON payload
    data = request.get_json()

    # Extract the serialized loss value from the payload
    serialized_loss_value = data['loss_value']

    # Deserialize the loss value
    loss_value = pickle.loads(serialized_loss_value.encode('latin1'))

    optimizer.zero_grad()
    print(f'V-MPO Loss value: {loss_value.item()}')

    # Create the tensor for backpropagation
    loss_tensor = torch.tensor(loss_value.item(), requires_grad=True)

    # Perform backpropagation
    loss_tensor.backward()
    print('Backward step done')

    # Perform optimizer step
    optimizer.step()
    print('Optimizer step done')

    return jsonify({'backward': 'ok'})


@app.route('/infer_actor_critic')
def make_inference_actor_critic():
    print(f'Got input actor critic')
    data = request.get_json()
    instruction_input = data['instruction_input']
    cc_net5_payload_numpy = json.loads(instruction_input)
    cc_net5_payload = {
        key: torch.tensor(value) if isinstance(value, list) else value
        for key, value in cc_net5_payload_numpy.items()
    }
    rgb_data = cc_net5_payload['rgb_input'].to(device)
    language_data = cc_net5_payload['language_input'].to(device)
    previous_action = cc_net5_payload['previous_action'].to(device)
    t5_output = cc_net5_payload['t5_output'].to(device)

    prediction = make_inference_model(actor_critic, rgb_data, language_data, previous_action, t5_output)
    print(f'prediction: {prediction.shape}')

    return jsonify({'prediction': f'{prediction}'})

@app.route('/infer_actor_normal', methods=['POST'])
def infer_actor_normal():
    print(f'Got input actor normal')
    data = request.get_json()
    instruction_input = data['instruction_input']
    cc_net5_payload_numpy = json.loads(instruction_input)
    cc_net5_payload = {
        key: torch.tensor(value) if isinstance(value, list) else value
        for key, value in cc_net5_payload_numpy.items()
    }
    rgb_data = cc_net5_payload['rgb_input'].to(device)
    language_data = cc_net5_payload['language_input'].to(device)
    previous_action = cc_net5_payload['previous_action'].to(device)
    t5_output = cc_net5_payload['t5_output'].to(device)

    prediction = make_inference_model(multimodal, rgb_data, language_data, previous_action, t5_output)
    print(f'prediction: {prediction.shape}')

    serialized_loss_value = pickle.dumps(prediction).decode('latin1')

    # Convert the payload to JSON
    json_payload = json.dumps(serialized_loss_value)

    return jsonify({'prediction': f'{prediction}'})


@app.route('/', methods=['POST'])
def make_inference():
    print('got input')
    data = request.get_json()
    #parsed_data = json.loads(data)
    instruction_input = data['instruction_input']
    cc_net5_payload_numpy = json.loads(instruction_input)
    cc_net5_payload = {
        key: torch.tensor(value) if isinstance(value, list) else value
        for key, value in cc_net5_payload_numpy.items()
    }
    rgb_data = cc_net5_payload['rgb_input'].to(device)
    language_data = cc_net5_payload['language_input'].to(device)
    previous_action = cc_net5_payload['previous_action'].to(device)
    t5_output = cc_net5_payload['t5_output'].to(device)

    print(f'rgb_data: {rgb_data.shape}, language_data: {language_data.shape}, previous_action: {previous_action.shape}, t5_output: {t5_output.shape}')
    print(f'rgb_data: {rgb_data.dtype}, language_data: {language_data.dtype}, previous_action: {previous_action.dtype}, t5_output: {t5_output.dtype}')

    prediction = make_inference_model(multimodal, rgb_data, language_data, previous_action, t5_output)
    print(f'prediction: {prediction.shape}')

    formatted_output = decode_output(tokenizer, prediction.to('cpu').detach())
    print(f'formatted_output: {formatted_output}')

    print('about to return request')
    #prediction = "e"
    return jsonify({'prediction': f'{prediction}'})

In [ ]:
app.run(port=80)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:80
INFO:werkzeug:Press CTRL+C to quit


Got input actor normal
torch.float32 torch.float32 torch.float32 torch.float32


INFO:werkzeug:127.0.0.1 - - [12/Jun/2023 17:13:49] "POST /infer_actor_normal HTTP/1.1" 200 -


action_output: torch.Size([1, 577]) - torch.float32
prediction: torch.Size([1, 577])
Got input actor normal
torch.float32 torch.float32 torch.float32 torch.float32


...

action_output: torch.Size([1, 577]) - torch.float32
prediction: torch.Size([1, 577])
Got input actor normal
torch.float32 torch.float32 torch.float32 torch.float32


INFO:werkzeug:127.0.0.1 - - [12/Jun/2023 17:37:54] "POST /infer_actor_normal HTTP/1.1" 200 -


action_output: torch.Size([1, 577]) - torch.float32
prediction: torch.Size([1, 577])
Got input actor normal
torch.float32 torch.float32 torch.float32 torch.float32


INFO:werkzeug:127.0.0.1 - - [12/Jun/2023 17:37:58] "POST /infer_actor_normal HTTP/1.1" 200 -


action_output: torch.Size([1, 577]) - torch.float32
prediction: torch.Size([1, 577])


In [ ]:
tokenizer.stoi['<tr'], tokenizer.itos[85], tokenizer.itos[1235]

(1235, '100', '<tr')

# Test encoding / decoding with embeddings

Tokenize input string

In [ ]:
input_string = '999'
stoi_value = tokenizer.stoi[input_string]

Embed ref test from Index of Vocab

In [ ]:
embedded_ref = torch.flatten(tokenizer.embedding_fn(torch.from_numpy(np.array(stoi_value)))).detach().numpy()
# Here is our emebeding
embedded_ref.shape, embedded_ref

((64,),
 array([ 7.24650562e-01,  1.55465722e+00,  9.79292035e-01, -7.00678170e-01,
        -2.37068677e+00, -9.89872515e-01, -6.16731167e-01,  1.13634884e+00,
         5.96476018e-01,  1.28716552e+00, -1.10430336e-02, -1.47615170e+00,
         1.65801823e-01, -9.02195334e-01, -1.90470129e-01, -1.43564057e+00,
         7.20837116e-01,  1.77453950e-01, -1.34255123e+00, -1.64361286e+00,
        -1.30364799e+00, -1.52187538e+00,  3.07258189e-01,  1.15209270e+00,
        -1.48790979e+00,  1.99890882e-01,  1.24265063e+00, -1.74484581e-01,
        -1.32448256e+00,  1.70079029e+00, -2.98048168e-01, -5.27164459e-01,
         8.16283524e-01,  1.14198291e+00,  8.19505036e-01, -1.40216017e+00,
        -3.02055657e-01, -6.24877691e-01, -2.18009889e-01,  1.17287941e-01,
        -1.02686810e+00, -3.83700972e-04,  1.25882804e+00, -2.72618860e-01,
        -1.12034810e+00, -1.29008055e+00, -5.15826881e-01, -9.08787847e-01,
         3.38234842e-01, -1.05625235e-01,  1.46444297e+00, -5.51725745e-01,
    

In [ ]:
# Recover the original index from the embeddings
decoded_ref_test, similarity_scores = find_closest_embedding(embedded_ref, tokenizer.embedding_fn.embedding.weight.detach().numpy())
decoded_ref_test

1148

Now decode it with itos

In [ ]:
tokenizer.itos[decoded_ref_test], input_string == tokenizer.itos[decoded_ref_test]

('999', True)

# Test payload

In [ ]:
import json

cc_net5_payload = {
    'rgb_input': torch.randn(1, 3, 160, 210),
    'language_input': torch.randn(512, 64),
    'previous_action': torch.randn(1, 577),
    't5_output': torch.randn(1, 577),
}

cc_net5_payload_numpy = {
        key: value.detach().numpy().tolist() if isinstance(value, torch.Tensor) else value
        for key, value in cc_net5_payload.items()
    }

json_data = json.dumps(cc_net5_payload_numpy)

instruction_input = {'instruction_input': json_data}
request = json.dumps(instruction_input)
print(request)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
parsed_data = json.loads(request)
instruction_input = parsed_data['instruction_input']
cc_net5_payload_numpy = json.loads(instruction_input)
cc_net5_payload = {
    key: torch.tensor(value) if isinstance(value, list) else value
    for key, value in cc_net5_payload_numpy.items()
}
rgb_input = cc_net5_payload['rgb_input']
language_input = cc_net5_payload['language_input']
previous_action = cc_net5_payload['previous_action']
t5_output = cc_net5_payload['t5_output']
